<a href="https://colab.research.google.com/github/kk412027247/nlp/blob/main/generating_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.wangluoguimi.com/news/news-headlines.tsv